In [ ]:
from pathlib import Path
import pandas as pd
import ee
import geemap
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

ee.Initialize()

In [ ]:
from ipyleaflet import Marker, MarkerCluster

In [ ]:
def get_lon_lat(plot_id, gold):

    # Get lon of particular plot
    lon = gold[gold["pl_plotid"] == plot_id]["lon"].iloc[0]
    lat = gold[gold["pl_plotid"] == plot_id]["lat"].iloc[0]

    return lon, lat


def get_map(point, zoom=15):
    # Set up interactive map.
    Map = geemap.Map(lite_mode=False)
    Map.add_basemap("SATELLITE")
    Map.center_object(point, zoom)

    return Map

def get_jaxa_layer():
    fnf = jaxa.select("fnf")
    vis = {"min": 1.0, "max": 3.0, "palette": ["006400", "FEFF99", "0000FF"]}
    return fnf, vis

In [ ]:
def get_composite_layer(point):
    region_around_point = point.buffer(500)
    # collection2000 = (
    #     ee.ImageCollection("LANDSAT/LE07/C01/T1_RT")
    #     .filterDate("2000-01-01", "2000-12-31")
    #     .filterBounds(point)
    # )
    # collection2010 = (
    #     ee.ImageCollection("LANDSAT/LE07/C01/T1_RT")
    #     .filterDate("2010-01-01", "2010-12-31")
    #     .filterBounds(point)
    # )
    collection2018 = (
        ee.ImageCollection("LANDSAT/LE07/C01/T1_RT")
        .filterDate("2018-01-01", "2018-12-31")
        .filterBounds(region_around_point)
    )

    # collection2000 = ee.Algorithms.Landsat.simpleComposite(collection2000)
    # collection2010 = ee.Algorithms.Landsat.simpleComposite(collection2010)
    collection2018 = ee.Algorithms.Landsat.simpleComposite(collection2018)

    # Map.addLayer(collection2000.clip(point), {bands:["B7","B5","B3"], max:128}, "real 2000-753")
    # Map.addLayer(collection2010.clip(point), {bands:["B7","B5","B3"], max:128}, "real 2010-753")

    return collection2018.clip(region_around_point)

In [ ]:
jaxa = collection = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/FNF").filterDate(
    "2017-01-01", "2017-12-31"
)
hexas = ee.FeatureCollection("users/jjaakko/Finland_776_Hexagons_")

In [ ]:
gold = pd.read_csv(Path("results/training_complete.csv"))

# Hexa visualizer

Render satellite view, jaxa 2017 and landsat 7 composite from 2018 for the given hexagon.

Some plotids to try out:  
- 524872647
- 525122005
- 523291679 

In [ ]:
button = widgets.Button(description="Show hexagon")
output = widgets.Output()
plotID = widgets.Text(
    value="524872647",
    placeholder="",
    description="Enter hexagon's plotid:",
    style=dict(description_width="initial"),
    disabled=False,
)
display(plotID, button)

In [ ]:
display(output)

In [ ]:
output2 = widgets.Output()
display(output2)

In [ ]:
def button_clicked(b):
    with output:
        try:
            lon, lat = get_lon_lat(int(plotID.value), gold)
            point = ee.Geometry.Point([lon, lat])
            Map = get_map(point)
            
            fnf, vis = get_jaxa_layer()
            # Show jaxa forest vs non-forest.
            Map.addLayer(fnf, vis, "fnf", True)
            
            composite = get_composite_layer(point)
            Map.addLayer(
                composite,
                {"bands": ["B7", "B5", "B3"], "max": 128},
                "real 2018-753",
            )
            # Show hexagons on top.
            Map.addLayer(hexas, {"color": "blue"}, "hexas", True, 0.3)
            clear_output()
            display(HTML(f"Displaying {plotID.value}"))
            display(Map)
        except IndexError:
            clear_output()
            display(HTML(f"Hexagon with plotID of {plotID.value} not found."))
        except ValueError:
            clear_output()
            display(HTML(f"PlotID has to be an integer value."))
            
    with output2:
        try:
            lon, lat = get_lon_lat(int(plotID.value), gold)
            point = ee.Geometry.Point([lon, lat])
            Map2 = get_map(point, zoom=6)
            composite2 = get_composite_layer(point)
            Map2.addLayer(
                composite2,
                {"bands": ["B7", "B5", "B3"], "max": 128},
                "real 2018-753", False
            )
            # Show hexagons on top.
            Map2.addLayer(hexas, {"color": "blue"}, "hexas", True, 0.3)
            markers = [Marker(location=[lat,lon])]
            marker_cluster = MarkerCluster(
                markers=markers,
                name = 'Markers')
            Map2.add_layer(marker_cluster)
            clear_output()
            display(HTML(f"Displaying {plotID.value}"))
            display(Map2)
        except IndexError:
            clear_output()
            display(HTML(f"Hexagon with plotID of {plotID.value} not found."))
        except ValueError:
            clear_output()
            display(HTML(f"PlotID has to be an integer value."))

button.on_click(button_clicked)